## [CSI Process Workflow] Global Shelf [Creating the GLOBAL, ARCTIC, and ANTARCTIC combined or GROUPED shelf class models]
<br>
Notebook Scope:
- Create a GROUPED (all classes combined) shelf model for geomorphon-form shelf for the GLOBAL, ARCTIC, and ANTARCTIC regions
- Convert raster models to vector layers
- Export vector shelf models to external ESRI Shapefiles

NOTE: models for bathymetry and slope are not (yet) included in this workflow. 

Notebook details processes to combine the four depth classes (shallow, intermediate, deep, and para) into a single, combined and classified (i.e. GROUPED) raster shelf model. Classification is as follows:

1 -> shallow shelf <br>
2 -> intermediate shelf <br>
3 -> deep shelf <br>
4 -> parashelf


#### The Base Geomorphon Model:
The model is defined using the geomorphon 'flat' form where 'flat' is defined as slopes that do not exceed 0.69°. Further, depths are constrained (based on exploratory clustering) to four depth windows: 


#### The Shelf Depth Classes:

- (shallow)  0 meters to -166.8 meters 
- (intermediate) -168.3 meters to -420.2 meters 
- (deep) -422.9 meters to -742.9 meters 
- (para-shelf, shelf-like) -742.9 meters down to -1,211 meters

##### Notes:
- All data here is projected in the interrupted Goode Homolosine Projection (the IGH is an Equal Area projection)
- flatness = 0° to 0.69°
- search=5  (maximum window size-so that we don't lose too much of the small shelf bits)
- distance=0
- extract from the 1km resolution ETOPO1 topobathy DEM
- Region Extents: xmin=-20015000, ymin=-8673000,xmax= 20015000, ymax=8673000
- can use the ETOPO1_World_Shelf_Zone(k) layers, where k is the zone number from 1 thru 13, as masks for analysis

##### The k=13 Zones:
|1   | Eastern North America <br> 
|2	 | Western North America <br>
|3	 | Eastern South America <br>
|4	 | Western South America <br>
|5	 | Europe <br>
|6	 | Western Africa <br>
|7	 | Indian/Oceana <br>
|8	 | Eastern Asia <br>
|9	 | Australia <br>
|10	 | Arctic <br>
|11	 | Antarctic <br>
|12	 | Mediterranean <br> 
|13	 | New Zealand (not currently included in the analysis)

#### Declare the class definition used to initialized the GRASS 7.1 instances needed for the processing to come (an instance allows calling GRASS libraries from outside of a GRASS session--cool eh?)

In [1]:
# This class definition is used to initialize an instance of GRASS 7.x in this domain space
class GrassBASE:
    def initGRASSEnv( self, database, loc, map ):
        gisbase=os.environ[ 'GISBASE' ] = '/Applications/GRASS-7.1.app/Contents/MacOS'
        gisdbase=database
        location=loc
        mapset=map
        sys.path.append(os.path.join(os.environ['GISBASE'], "etc", "python"))
        import grass.script as gscript
        import grass.script.setup as gsetup
        gsetup.init( gisbase, gisdbase, location, mapset )
        print(gscript.gisenv())
        print('GRASS Application base path:',gisbase)

### Create a GROUPED GLOBAL shelf surface model by (re)combining the four shelf depth class rasters 

The 4 shelf-depth class geomorphon models are here recombined, but this time we retain the differentiation between class by assigning, encoding, the new grouped raster based on class. The key works simply:

1 -> shallow shelf <br>
2 -> intermediate shelf <br>
3 -> deep shelf <br>
4 -> parashelf

This is done (in the script below) by:

#### 1.) change the name of the class raster for intermediate, deep, and para-shelf environments:
g.rename raster=ETOPO1_World_Shelf_intermed_1km,tmp_intermed
g.rename raster=ETOPO1_World_Shelf_deep_1km,tmp_deep
g.rename raster=ETOPO1_World_Shelf_parashelf_1km,tmp_parashelf

Note: we need not change the shallow layer as it is already encoded as equal to 1


#### 2.) reclass the intermediate, deep, and para-shelf model rasters:
r.mapcalc —o
ETOPO1_World_Shelf_intermed_1km=if(tmp_intermed, 2, null() )

r.mapcalc —o
ETOPO1_World_Shelf_deep_1km=if(tmp_deep, 3, null() )

r.mapcalc —o
ETOPO1_World_Shelf_parashelf_1km=if(tmp_parashelf, 4, null() )


#### 3.) delete the temporary shelf rasters (clean up):
g.remove -f type=raster name=tmp_intermed,tmp_deep,tmp_parashelf


#### 4.) combine (patch) the group of four into one…
r.patch —o input=ETOPO1_World_Shelf_shallow_1km,ETOPO1_World_Shelf_intermed_1km,ETOPO1_World_Shelf_deep_1km,ETOPO1_World_Shelf_parashelf_1km output=ETOPO1_World_Shelf_grouped_1km

### Combine the four shelf class raster [GEOMORPHON] models for the GLOBAL data into one GROUPED model: 
Note that the GLOBAL data do not include the polar Arctic and Anarctic regions! See further along in the notebook for coverage of these latter (chilly) zones...

In [2]:
import os, sys

# set the location and mapset for the GRASS instance to be created
gisdbase = '/Users/paulp/Documents/csi/cshelves/data/GRASSData'    # location of GRASS database  
location = 'csi_shelf_mapping_global'                              # name of location
mapset = 'user'                                                    # name of mapset containing data


# initiate a GRASS instance/environment

print('')
print('********************************************')
print('Setting up a GRASS 7.1 Environment') 
g = GrassBASE()
g.initGRASSEnv( gisdbase, location, mapset )
#print(sys.path)
print()
print(os.environ[ 'DYLD_LIBRARY_PATH' ])


********************************************
Setting up a GRASS 7.1 Environment
{u'MAPSET': u'user', u'GISDBASE': u'/Users/paulp/Documents/csi/cshelves/data/GRASSData', u'LOCATION_NAME': u'csi_shelf_mapping_global'}
('GRASS Application base path:', '/Applications/GRASS-7.1.app/Contents/MacOS')
()
:/Applications/GRASS-7.1.app/Contents/MacOS/lib


In [3]:
import grass.script as gscript

# ## set the working geographic region:
gscript.run_command( 'g.region', flags='p', region='ETOPO1_World_1km')

# ## rename the intermediate, deep, and para-shelf models (we assume that they already exist):
gscript.run_command( 'g.rename', raster='ETOPO1_World_Shelf_intermed_1km,tmp_intermed' )
gscript.run_command( 'g.rename', raster='ETOPO1_World_Shelf_deep_1km,tmp_deep' )
gscript.run_command( 'g.rename', raster='ETOPO1_World_Shelf_parashelf_1km,tmp_parashelf' )

# ## do the reclassification for the three shelf class models:
gscript.mapcalc('ETOPO1_World_Shelf_intermed_1km=if(tmp_intermed, 2, null() )', overwrite=True )
gscript.mapcalc('ETOPO1_World_Shelf_deep_1km=if(tmp_deep, 3, null() )', overwrite=True )
gscript.mapcalc('ETOPO1_World_Shelf_parashelf_1km=if(tmp_parashelf, 4, null() )', overwrite=True )

# ## combine the 4 depth class models, now reclassified to allow for discrimination between each class, into a 
# ## single grouped model:
gscript.run_command( 'r.patch', overwrite=True, input='ETOPO1_World_Shelf_shallow_1km,ETOPO1_World_Shelf_intermed_1km,ETOPO1_World_Shelf_deep_1km,ETOPO1_World_Shelf_parashelf_1km', \
                    output='ETOPO1_World_Shelf_GROUPED_1km')

# ## clean up:
gscript.run_command('g.remove', flags='f', type='raster', name='tmp_intermed,tmp_deep,tmp_parashelf' )

0

### Create a GROUPED GLOBAL VECTOR model:

In [4]:
import grass.script as gscript

# ## set the working geographic region:
gscript.run_command( 'g.region', flags='p', region='ETOPO1_World_1km')

# ## create the vector layer:
gscript.run_command('r.to.vect', overwrite=True, input='ETOPO1_World_Shelf_GROUPED_1km', \
                    output='ETOPO1_GLOBAL_Shelf_GROUPED', type='area', column='CLASS' )

print('Fin!')

Fin!


### Export the GROUPED GLOBAL VECTOR model to an external ESRI Shapefile

In [5]:
import grass.script as gscript

# ## set the working geographic region:
gscript.run_command( 'g.region', flags='p', region='ETOPO1_World_1km')

gscript.run_command('v.out.ogr', overwrite=True, input='ETOPO1_GLOBAL_Shelf_GROUPED', \
                    output='ETOPO1_GLOBAL_Shelf_GROUPED', format='ESRI_Shapefile')

print('Fin!')

Fin!


----------------------------------------------------------------------------------------------------------------------

### Combine the four shelf class raster [GEOMORPHON] models for the ARCTIC data into one GROUPED model: 

In [6]:
# set the location and mapset for the GRASS instance to be created
gisdbase = '/Users/paulp/Documents/csi/cshelves/data/GRASSData'    # location of GRASS database  
location = 'CSM_Arctic_Region'                                     # name of location
mapset = 'user'                                                    # name of mapset containing data


# initiate a GRASS instance/environment

print('')
print('********************************************')
print('Setting up a GRASS 7.1 Environment') 
g = GrassBASE()
g.initGRASSEnv( gisdbase, location, mapset )
#print(sys.path)
print()
print(os.environ[ 'DYLD_LIBRARY_PATH' ])


********************************************
Setting up a GRASS 7.1 Environment
{u'MAPSET': u'user', u'GISDBASE': u'/Users/paulp/Documents/csi/cshelves/data/GRASSData', u'LOCATION_NAME': u'CSM_Arctic_Region'}
('GRASS Application base path:', '/Applications/GRASS-7.1.app/Contents/MacOS')
()
:/Applications/GRASS-7.1.app/Contents/MacOS/lib:/Applications/GRASS-7.1.app/Contents/MacOS/lib


In [7]:
import grass.script as gscript

# ## set the working geographic region:
gscript.run_command( 'g.region', flags='p', region='ETOPO1_Arctic_1km')

# ## rename the intermediate, deep, and para-shelf models (we assume that they already exist):
gscript.run_command( 'g.rename', raster='ETOPO1_Arctic_Shelf_intermed_1km,tmp_intermed' )
gscript.run_command( 'g.rename', raster='ETOPO1_Arctic_Shelf_deep_1km,tmp_deep' )
gscript.run_command( 'g.rename', raster='ETOPO1_Arctic_Shelf_parashelf_1km,tmp_parashelf' )

# ## do the reclassification for the three shelf class models:
gscript.mapcalc('ETOPO1_Arctic_Shelf_intermed_1km=if(tmp_intermed, 2, null() )', overwrite=True )
gscript.mapcalc('ETOPO1_Arctic_Shelf_deep_1km=if(tmp_deep, 3, null() )', overwrite=True )
gscript.mapcalc('ETOPO1_Arctic_Shelf_parashelf_1km=if(tmp_parashelf, 4, null() )', overwrite=True )

# ## combine the 4 depth class models, now reclassified to allow for discrimination between each class, into a 
# ## single grouped model:
gscript.run_command( 'r.patch', overwrite=True, input='ETOPO1_Arctic_Shelf_shallow_1km,ETOPO1_Arctic_Shelf_intermed_1km,ETOPO1_Arctic_Shelf_deep_1km,ETOPO1_Arctic_Shelf_parashelf_1km', \
                    output='ETOPO1_ARCTIC_Shelf_GROUPED_1km')

# ## clean up:
gscript.run_command('g.remove', flags='f', type='raster', name='tmp_intermed,tmp_deep,tmp_parashelf' )

print('Fin!')

Fin!


### Create a GROUPED ARCTIC VECTOR model:

In [8]:
import grass.script as gscript

# ## set the working geographic region:
gscript.run_command( 'g.region', flags='p', region='ETOPO1_Arctic_1km')

# ## create the vector layer:
gscript.run_command('r.to.vect', overwrite=True, input='ETOPO1_ARCTIC_Shelf_GROUPED_1km', \
                    output='ETOPO1_ARCTIC_Shelf_GROUPED', type='area', column='CLASS' )

print('Fin!')

Fin!


### Export the GROUPED ARCTIC VECTOR model to an external ESRI Shapefile

In [9]:
import grass.script as gscript

# ## set the working geographic region:
gscript.run_command( 'g.region', flags='p', region='ETOPO1_Arctic_1km')

gscript.run_command('v.out.ogr', overwrite=True, input='ETOPO1_ARCTIC_Shelf_GROUPED', \
                    output='ETOPO1_ARCTIC_Shelf_GROUPED', format='ESRI_Shapefile')

print('Fin!')

Fin!


----------------------------------------------------------------------------------------------------------------------

### Combine the four shelf class raster [GEOMORPHON] models for the ANTARCTIC data into one GROUPED model: 

In [10]:
# set the location and mapset for the Antarctic GRASS instance to be created
gisdbase = '/Users/paulp/Documents/csi/cshelves/data/GRASSData'    # location of GRASS database  
location = 'CSM_Antarctic_Region'                                     # name of location
mapset = 'user'                                                    # name of mapset containing data

# initiate a GRASS instance/environment

print('')
print('********************************************')
print('Setting up a GRASS 7.1 Environment') 
g = GrassBASE()
g.initGRASSEnv( gisdbase, location, mapset )
#print(sys.path)
print()
print(os.environ[ 'DYLD_LIBRARY_PATH' ])


********************************************
Setting up a GRASS 7.1 Environment
{u'MAPSET': u'user', u'GISDBASE': u'/Users/paulp/Documents/csi/cshelves/data/GRASSData', u'LOCATION_NAME': u'CSM_Antarctic_Region'}
('GRASS Application base path:', '/Applications/GRASS-7.1.app/Contents/MacOS')
()
:/Applications/GRASS-7.1.app/Contents/MacOS/lib:/Applications/GRASS-7.1.app/Contents/MacOS/lib:/Applications/GRASS-7.1.app/Contents/MacOS/lib


In [11]:
import grass.script as gscript

# ## set the working geographic region:
gscript.run_command( 'g.region', flags='p', region='ETOPO1_Antarctic_1km')

# ## rename the intermediate, deep, and para-shelf models (we assume that they already exist):
gscript.run_command( 'g.rename', raster='ETOPO1_Antarctic_Shelf_intermed_1km,tmp_intermed' )
gscript.run_command( 'g.rename', raster='ETOPO1_Antarctic_Shelf_deep_1km,tmp_deep' )
gscript.run_command( 'g.rename', raster='ETOPO1_Antarctic_Shelf_parashelf_1km,tmp_parashelf' )

# ## do the reclassification for the three shelf class models:
gscript.mapcalc('ETOPO1_Antarctic_Shelf_intermed_1km=if(tmp_intermed, 2, null() )', overwrite=True )
gscript.mapcalc('ETOPO1_Antarctic_Shelf_deep_1km=if(tmp_deep, 3, null() )', overwrite=True )
gscript.mapcalc('ETOPO1_Antarctic_Shelf_parashelf_1km=if(tmp_parashelf, 4, null() )', overwrite=True )

# ## combine the 4 depth class models, now reclassified to allow for discrimination between each class, into a 
# ## single grouped model:
gscript.run_command( 'r.patch', overwrite=True, input='ETOPO1_Antarctic_Shelf_shallow_1km,ETOPO1_Antarctic_Shelf_intermed_1km,ETOPO1_Antarctic_Shelf_deep_1km,ETOPO1_Antarctic_Shelf_parashelf_1km', \
                    output='ETOPO1_ANTARCTIC_Shelf_GROUPED_1km')

# ## clean up:
gscript.run_command('g.remove', flags='f', type='raster', name='tmp_intermed,tmp_deep,tmp_parashelf' )

print('Fin!')

Fin!


### Create a GROUPED ANTARCTIC VECTOR model:

In [12]:
import grass.script as gscript

# ## set the working geographic region:
gscript.run_command( 'g.region', flags='p', region='ETOPO1_Antarctic_1km')

# ## create the vector layer:
gscript.run_command('r.to.vect', overwrite=True, input='ETOPO1_ANTARCTIC_Shelf_GROUPED_1km', \
                    output='ETOPO1_ANTARCTIC_Shelf_GROUPED', type='area', column='CLASS' )

print('Fin!')

Fin!


### Export the GROUPED ANTARCTIC VECTOR model to an external ESRI Shapefile

In [13]:
import grass.script as gscript

gscript.run_command('v.out.ogr', overwrite=True, input='ETOPO1_ANTARCTIC_Shelf_GROUPED', \
                    output='ETOPO1_ANTARCTIC_Shelf_GROUPED', format='ESRI_Shapefile')

print('Fin!')

Fin!


### You can add code to build GROUPED BATHY and SLOPE-based models to accompany the GEOMORPHONs, if needed, here.